In [1]:
import os
os.chdir("../src")

In [2]:
from sqlmodel import Session, select
from api.common.dependency_container import DependencyContainer
from domain.entities import subject, topic, section, summary

DC = DependencyContainer()
DC.initialize()

def select_table(type_class):
    with Session(DC._database_engine) as session:
        statement = select(type_class)
        results = session.exec(statement)
        return results.all()
    
subjects_table = select_table(subject.Subject)
topic_table = select_table(topic.Topic)
section_table = select_table(section.Section)
summaries_table = select_table(summary.Summary)

In [4]:
import gradio as gr
from api.workflows.test_questions.test_request import TestRequest
from api.common.dependency_container import DependencyContainer

subjects_dropped_list = [x.subject_name for x in subjects_table]

DC = DependencyContainer()
DC.initialize()

def select_table(type_class):
    with Session(DC._database_engine) as session:
        statement = select(type_class)
        results = session.exec(statement)
        return results.all()

def greet(Content, prompt_pregunta):
    
    request = TestRequest(text= Content, prompt= prompt_pregunta)
    
    return str(DC.get_text_test_workflow().execute(request))


def subject_change(rs):
    
    subject_id = [x.subject_id for x in subjects_table if x.subject_name == rs]
    topic_dropped_list = [x.topic_name for x in topic_table if x.subject_id == subject_id[0]]
    
    return gr.update(choices=topic_dropped_list, value=None)

def topic_change(rs):
    if rs:
        topic_id = [x.topic_id for x in topic_table if x.topic_name == rs]
        section_dropped_list = [x.section_name for x in section_table if x.topic_id == topic_id[0]]

        return gr.update(choices=section_dropped_list, value=None)
    else:
        return gr.update(choices=["no topic selected"], value=None)
    
def section_change(rs):
    if rs:
        section_id = [x.section_id for x in section_table if x.section_name == rs]
        content = [x.content for x in section_table if x.section_name == rs]
    
    
with gr.Blocks() as demo:

    drop_subject = gr.Dropdown(subjects_dropped_list, label="subject")
    drop_topic = gr.Dropdown([], label="Topic", interactive=True)
    drop_section = gr.Dropdown([], label="Section", interactive=True)
    
    
    summary = gr.Textbox(lines=1, label="Topic Summary")
    
    gr.Interface(
    fn=greet,
    inputs=[
            gr.Textbox(lines=1, label="No va a estar"),
            gr.Textbox(lines=1, label="Prompt"),
            ],
    
    outputs="text",)

    drop_subject.change(fn=subject_change, inputs=[drop_subject], outputs=[drop_topic])
    drop_topic.change(fn=topic_change, inputs=[drop_topic], outputs=[drop_section])

demo.launch(share=True)

c:\Users\juanjose.rueda\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


ERROR:asyncio:Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\juanjose.rueda\AppData\Local\Programs\Python\Python311\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\juanjose.rueda\AppData\Local\Programs\Python\Python311\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto
